In [51]:
from lark import Lark

with open ("./foreduce/tptp/tptp_fof_cnf.lark", "r") as f:
    text = f.read()
    text.replace("<", "").replace(">", "").replace("::=", ":")

lexer = Lark(text, start="tptp_file")

In [2]:
import os
from tqdm import tqdm

results = []
success, failures = [], []
tptp_path = "/home/apluska/TPTP-v8.2.0/"
for file in tqdm(os.listdir(tptp_path + "Problems/PUZ/")):
    with open(tptp_path + "Problems/PUZ/" + file, "r") as f:
        data = f.read()
    try:
        result = lexer.parse(data)
    except Exception as e:
        failures.append(file)
        continue
    success.append(file)
    results.append(result)
len(results)

  0%|          | 0/243 [00:00<?, ?it/s]

100%|██████████| 243/243 [00:38<00:00,  6.28it/s]


153

In [34]:
example = lexer.parse("""
fof(4, axiom,  ~$true).
""")
example.children[0].children[0].children[0].children[2].children[0]

Tree('fof_logic_formula', [Tree('fof_unitary', [Tree('fof_negation', [Tree('fof_unitary', [Tree('fof_atom', [Token('DEFINED_UNARY_PREDICATE', '$true')])])])])])

In [11]:
from foreduce.fol.logic import *

In [22]:
from lark import Tree, Token
from lark.visitors import Transformer_InPlace

class OpSimplify(Transformer_InPlace):
    # p <= q -> q => p
    # p <~> q -> ~(p <=> q)
    # p ~| q -> ~(p | q)
    # p ~& q -> ~(p & q)
    def fof_binary(self, children):
        match children[1].value:
            case "<=":
                return Tree("fof_binary", [children[2], Token("BINARY_CONNECTIVE", "=>"), children[0]])
            case "<~>":
                return Tree("fof_unary", [Tree("fof_negation", [Tree("fof_unary", [Tree("fof_logic_formula", [Tree("fof_binary", [children[0], Token("BINARY_CONNECTIVE", "<=>"), children[2]])])])])])
            case "~|":
                return Tree("fof_unary", [Tree("fof_negation", [Tree("fof_unary", [Tree("fof_logic_formula", [Tree("fof_binary", [children[0], Token("BINARY_CONNECTIVE", "|"), children[2]])])])])])
            case "~&":
                return Tree("fof_unary", [Tree("fof_negation", [Tree("fof_unary", [Tree("fof_logic_formula", [Tree("fof_binary", [children[0], Token("BINARY_CONNECTIVE", "&"), children[2]])])])])])
        return Tree("fof_binary", children)

transformer = OpSimplify()
transformer.transform(example)
print(example.pretty())

tptp_file
  tptp_input
    formula
      fof
        4
        axiom
        fof_formula
          fof_logic_formula
            fof_unitary
              fof_negation
                fof_unitary
                  fof_logic_formula
                    fof_binary
                      fof_unitary
                        fof_atom	p
                      <=>
                      fof_unitary
                        fof_atom	q
  tptp_input
    formula
      fof
        5
        axiom
        fof_formula
          fof_logic_formula
            fof_unitary
              fof_negation
                fof_unitary
                  fof_logic_formula
                    fof_binary
                      fof_unitary
                        fof_atom	q
                      |
                      fof_unitary
                        fof_atom	r
  tptp_input
    formula
      fof
        6
        axiom
        fof_formula
          fof_logic_formula
            fof_unitary
              fof_negation


In [47]:
example = lexer.parse("""
fof(4, axiom,  p <=> $false).
""")

class Simplify(Transformer_InPlace):
    def __init__(self):
        self.unchanged = False

    def fof_negation(self, children):
        if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]):
            self.unchanged = False
            return Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])
        elif children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]):
            self.unchanged = False
            return Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])
        return Tree("fof_negation", children)

    def fof_binary(self, children):
        match children[1].value:
            case "|":
                if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]) or children[2] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]):
                    self.unchanged = False
                    return Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])])
                if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]):
                    self.unchanged = False
                    return children[2]
                if children[2] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]):
                    self.unchanged = False
                    return children[0]
            case "&":
                if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]) or children[2] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]):
                    self.unchanged = False
                    return Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])])
                if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]):
                    self.unchanged = False
                    return children[2]
                if children[2] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]):
                    self.unchanged = False
                    return children[0]
            case "=>":
                if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]):
                    self.unchanged = False
                    return children[2]
                if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]):
                    self.unchanged = False
                    return Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])])
                if children[2] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]):
                    self.unchanged = False
                    return Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])])
                if children[2] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]):
                    self.unchanged = False
                    return Tree("fof_unary", [Tree("fof_negation", [children[0]])])
            case "<=>":
                if children[0] == children[2]:
                    self.unchanged = False
                    return Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])])
                if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]):
                    self.unchanged = False
                    return children[2]
                if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]):
                    self.unchanged = False
                    return Tree("fof_unary", [Tree("fof_negation", [children[2]])])
                if children[2] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]):
                    self.unchanged = False
                    return children[0]
                if children[2] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]):
                    self.unchanged = False
                    return Tree("fof_unary", [Tree("fof_negation", [children[0]])])
        return Tree("fof_binary", children)
                
                
                
transformer = Simplify()
transformer.transform(example)
print(example.pretty())
transformer.unchanged

tptp_file
  tptp_input
    formula
      fof
        4
        axiom
        fof_formula
          fof_logic_formula
            fof_unary
              fof_negation
                fof_unary
                  fof_atom	p



True

In [ ]:
class NNF(Transformer_InPlace):
    def fof_negation(self, children):
        if children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])]):
            self.modified = True
            return Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])
        elif children[0] == Tree("fof_unary", [Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$true")])]):
            return Tree("fof_atom", [Token("DEFINED_UNARY_PREDICATE", "$false")])
        return Tree("fof_negation", children)
    
    